In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

I apologize in advance for how messy my notebook is, this is my first time publishing a notebook in Kaggle so bare with me.

In [ ]:
#After reading data we will use EDA to see distribution of data across different target classes.
data = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')

In [ ]:
# Use Pivot Tables to see 
print(pd.crosstab(data['blue'], data['price_range'], normalize=True)) # Bluetooth doesn't seem to be informative
print(pd.crosstab(data['dual_sim'], data['price_range'], normalize=True)) #Same with dual sim
print(pd.crosstab(data['four_g'], data['price_range'], normalize=True)) #same with 4 g
print(pd.crosstab(data['three_g'], data['price_range'], normalize=True)) #3 g looks helpful ~~~
print(pd.crosstab(data['touch_screen'], data['price_range'], normalize=True)) #touch screen a maybe |
print(pd.crosstab(data['wifi'], data['price_range'], normalize=True)) #wifi a no

In [ ]:
import seaborn as sns
#boxplots for the continous features
cont_features = ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 
                'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time' ]

# print(sns.boxplot(data['price_range'], data['battery_power']))# maybe /
# print(sns.boxplot(data['price_range'], data['clock_speed']))  # no
# print(sns.boxplot(data['price_range'], data['fc'])) # maybe
# print(sns.boxplot(data['price_range'], data['int_memory'])) # no
# print(sns.boxplot(data['price_range'], data['m_dep'])) # maybe
# print(sns.boxplot(data['price_range'], data['mobile_wt'])) # no
# print(sns.boxplot(data['price_range'], data['n_cores'])) # maybe
# print(sns.boxplot(data['price_range'], data['pc'])) # no
# print(sns.boxplot(data['price_range'], data['px_height'])) # maybe
# print(sns.boxplot(data['price_range'], data['px_width'])) # maybe
print(sns.boxplot(data['price_range'], data['ram'])) # yes /
# print(sns.boxplot(data['price_range'], data['sc_h'])) # no
# print(sns.boxplot(data['price_range'], data['sc_w'])) # no
# print(sns.boxplot(data['price_range'], data['talk_time'])) # no

possible_features = ['three_g', 'touch_screen', 'battery_power', 'fc', 
                     'm_dep', 'n_cores', 'px_height', 'px_width', 'ram', 'price_range']
df2 = data[possible_features]
#ram is clearly the most important feature

In [ ]:
#in this step we'll standardize the cont. columns and use PCA to extract the final features
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

cont_feat = ['battery_power', 'fc', 'n_cores', 'px_height', 'px_width', 'ram']
df2[cont_feat] = StandardScaler().fit_transform(df2[cont_feat])

In [ ]:
import matplotlib.pyplot as plt
X = df2.drop('price_range', axis=1)
y = df2['price_range']
model = PCA()
model.fit(X)
features = range(model.n_components_)
plt.bar(features, model.explained_variance_)
plt.xticks(features)
plt.xlabel('PCA Features')
plt.ylabel('Variance')
#The selected features seem to have an acceptable amount of variance so we will keep them

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77, test_size=.20)
knn = KNeighborsClassifier(n_neighbors=5)
dt = DecisionTreeClassifier

In [ ]:
#model Time!
from sklearn.ensemble import RandomForestClassifier
rfc =RandomForestClassifier(n_estimators=8,criterion="entropy",random_state=0)
knn = KNeighborsClassifier(n_neighbors=8)
dt = DecisionTreeClassifier(max_depth=9)
knn.fit(X_train, y_train)
print('Knn score: ', knn.score(X_test, y_test))
dt.fit(X_train, y_train)
print('Decision Tree score: ', dt.score(X_test, y_test))
rfc.fit(X_train, y_train)
print('Random Forest Score: ', rfc.score(X_test, y_test))

Thanks for reading all of this, any tips or criticism would be greatly appreciated!